# Lesson 2: Image Segmentation

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

* In this classroom, the libraries have been already installed for you.
* If you would like to run this code on your own machine, you need to install the following:
    ```
    !pip install ultralytics torch
    ```

### Load the sample image

In [ ]:
from PIL import Image
raw_image = Image.open("dogs.jpg")
raw_image

>Note: the images referenced in this notebook have already been uploaded to the Jupyter directory, in this classroom, for your convenience. For further details, please refer to the **Appendix** section located at the end of the lessons.

* Resize the image.

In [ ]:
from utils import resize_image
resized_image = resize_image(raw_image, input_size=1024)
resized_image

### Import and prepare the model

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Info about [torch](https://pytorch.org/).

In [ ]:
from ultralytics import YOLO
model = YOLO('./FastSAM.pt')

Info about ['FastSAM'](https://docs.ultralytics.com/models/fast-sam/)

### Use the model

>Note: ```utils``` is an additional file containing the methods that have been already developed for you to be used in this classroom. 
For further details, please refer to the **Appendix** section located at the end of the lessons.

In [ ]:
from utils import show_points_on_image

In [ ]:
# Define the coordinates for the point in the image
# [x_axis, y_axis]
input_points = [ [350, 450 ] ]

In [ ]:
input_labels = [1] # positive point

In [ ]:
# Function written in the utils file
show_points_on_image(resized_image, input_points)

In [ ]:
# Run the model
results = model(resized_image, device=device, retina_masks=True)

* Filter the mask based on the point defined before.

In [ ]:
from utils import format_results, point_prompt

In [ ]:
results = format_results(results[0], 0)

In [ ]:
# Generate the masks
masks, _ = point_prompt(results, input_points, input_labels)

In [ ]:
from utils import show_masks_on_image

In [ ]:
# Visualize the generated masks
show_masks_on_image(resized_image, [masks])

* Define 'semantic masks' - two points to be masked.

In [ ]:
# Specify two points in the same image
# [x_axis, y_axis]
input_points = [ [350, 450], [620, 450] ]

In [ ]:
# Specify both points as "positive prompt"
input_labels = [1 , 1] # both positive points

In [ ]:
# Visualize the points defined before
show_points_on_image(resized_image, input_points)

In [ ]:
# Run the model
results = model(resized_image, device=device, retina_masks=True)

In [ ]:
results = format_results(results[0], 0)

In [ ]:
# Generate the masks
masks, _ = point_prompt(results, input_points, input_labels)

In [ ]:
# Visualize the generated masks
show_masks_on_image(resized_image, [masks])

>Note: Please note that the results obtained from running this notebook may vary slightly from those demonstrated by the instructor in the video. 

* Identify subsections of the image by adding a **negative prompt**.

In [ ]:
# Define the coordinates for the regions to be masked
# [x_axis, y_axis]
input_points = [ [350, 450], [400, 300]  ]

In [ ]:
input_labels = [1, 0] # positive prompt, negative prompt

In [ ]:
# Visualize the points defined above
show_points_on_image(resized_image, input_points, input_labels)

>Note: From the image above, the red star indicates the negative prompt and the green star the positive prompt.

In [ ]:
# Run the model
results = model(resized_image, device=device, retina_masks=True)

In [ ]:
results = format_results(results[0], 0)

In [ ]:
# Generate the masks
masks, _ = point_prompt(results, input_points, input_labels)

In [ ]:
# Visualize the generated masks
show_masks_on_image(resized_image, [masks])

>Note: From the image above, only the jacket, from the dog in the left, was segmented, so, it is following the command given by the positive prompt!

### Prompting with bounding boxes

In [ ]:
from utils import box_prompt

In [ ]:
# Set the bounding box coordinates
# [xmin, ymin, xmax, ymax]
input_boxes = [530, 180, 780, 600]

In [ ]:
from utils import show_boxes_on_image

In [ ]:
# Visualize the bounding box defined with the coordinates above
show_boxes_on_image(resized_image, [input_boxes])

* Now, try to isolate the mask from the total output of the model.

In [ ]:
from utils import box_prompt

In [ ]:
results = model(resized_image, device=device, retina_masks=True)

In [ ]:
# Generate the masks
masks = results[0].masks.data

In [ ]:
masks

In [ ]:
# Convert to True/False boolean mask
masks = masks > 0

In [ ]:
masks

In [ ]:
masks, _ = box_prompt(masks, input_boxes)

In [ ]:
# Visualize the masks
show_masks_on_image(resized_image, [masks])

In [ ]:
# Print the segmentation mask, but in its raw format
masks

In [ ]:
# To visualize, import matplotlib
from matplotlib import pyplot as plt

In [ ]:
# Plot the binary mask as an image
plt.imshow(masks, cmap='gray')

### Try yourself! 
Try the image segmentation explained before with your own images.

In [ ]:
# To start opening images, upload your own or use the sample images we've uploaded, for example: younes.png
# The image younes.png is already uploaded in this classroom
raw_image = Image.open('younes.png')
raw_image

In [ ]:
# Resize image


In [ ]:
# Define the coordinates for the point: [x_axis, y_axis]


In [ ]:
# Define the positive or negative prompt


In [ ]:
# show_points_on_image(resized_image, input_points)

### Additional Resources

* For more on how to use [Comet](https://www.comet.com/site/?utm_source=dlai&utm_medium=course&utm_campaign=prompt_engineering_for_vision_models&utm_content=dlai_L2) for experiment tracking, check out this [Quickstart Guide](https://colab.research.google.com/drive/1jj9BgsFApkqnpPMLCHSDH-5MoL_bjvYq?usp=sharing) and the [Comet Docs](https://www.comet.com/docs/v2/?utm_source=dlai&utm_medium=course&utm_campaign=prompt_engineering_for_vision_models&utm_content=dlai_L2).
* This course was based off a set of two blog articles from Comet. Explore them here for more on how to use newer versions of Stable Diffusion in this pipeline, additional tricks to improve your inpainting results, and a breakdown of the pipeline architecture:
  * [SAM + Stable Diffusion for Text-to-Image Inpainting](https://www.comet.com/site/blog/sam-stable-diffusion-for-text-to-image-inpainting/?utm_source=dlai&utm_medium=course&utm_campaign=prompt_engineering_for_vision_models&utm_content=dlai_L2)
  * [Image Inpainting for SDXL 1.0 Base Model + Refiner](https://www.comet.com/site/blog/image-inpainting-for-sdxl-1-0-base-refiner/?utm_source=dlai&utm_medium=course&utm_campaign=prompt_engineering_for_vision_models&utm_content=dlai_L2)
